In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
from six.moves.urllib.request import urlopen

import tensorflow as tf
import numpy as np

/home/voyageth/develop/anaconda3/envs/kaggle/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
IRIS_TRAINING = "input/iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "input/iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

In [6]:
if not os.path.exists(IRIS_TRAINING):
    raw = urlopen(IRIS_TRAINING_URL).read()
    with open(IRIS_TRAINING, 'wb') as f:
        f.write(raw)

if not os.path.exists(IRIS_TEST):
    raw = urlopen(IRIS_TEST_URL).read()
    with open(IRIS_TEST, 'wb') as f:
        f.write(raw)

In [8]:
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TRAINING,
    target_dtype=np.int,
    features_dtype=np.float32
)

test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TEST,
    target_dtype=np.int,
    features_dtype=np.float32
)

In [9]:
feature_columns = [tf.feature_column.numeric_column('x', shape=[4])]

classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                       hidden_units=[10,20,10],
                                       n_classes=3,
                                       model_dir='output/iris_model')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'output/iris_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd31edb3588>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [10]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x':np.array(training_set.data)},
    y=np.array(training_set.target),
    num_epochs=None,
    shuffle=True
)

In [11]:
classifier.train(input_fn=train_input_fn, steps=2000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into output/iris_model/model.ckpt.
INFO:tensorflow:loss = 201.708, step = 1
INFO:tensorflow:global_step/sec: 568.342
INFO:tensorflow:loss = 11.9646, step = 101 (0.177 sec)
INFO:tensorflow:global_step/sec: 621.736
INFO:tensorflow:loss = 11.1204, step = 201 (0.161 sec)
INFO:tensorflow:global_step/sec: 626.392
INFO:tensorflow:loss = 8.63682, step = 301 (0.160 sec)
INFO:tensorflow:global_step/sec: 634.321
INFO:tensorflow:loss = 13.6762, step = 401 (0.158 sec)
INFO:tensorflow:global_step/sec: 636.269
INFO:tensorflow:loss = 8.18977, step = 501 (0.157 sec)
INFO:tensorflow:global_step/sec: 610.415
INFO:tensorflow:loss = 6.46853, step = 601 (0.164 sec)
INFO:tensorflow:global_step/sec: 636.763
INFO:tensorflow:loss = 7.06398, step = 701 (0.157 sec)
INFO:tensorflow:global_step/sec: 618.046
INFO:tensorflow:loss = 8.74011, step = 801 (0.162 sec)
INFO:tensorflow:global_step/sec: 638.735
INFO:tensorflow:loss = 3.6415,

In [12]:
classifier.train(input_fn=train_input_fn, steps=2000)
classifier.train(input_fn=train_input_fn, steps=2000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from output/iris_model/model.ckpt-2000
INFO:tensorflow:Saving checkpoints for 2001 into output/iris_model/model.ckpt.
INFO:tensorflow:loss = 8.34046, step = 2001
INFO:tensorflow:global_step/sec: 552.061
INFO:tensorflow:loss = 8.03179, step = 2101 (0.182 sec)
INFO:tensorflow:global_step/sec: 609.044
INFO:tensorflow:loss = 11.062, step = 2201 (0.164 sec)
INFO:tensorflow:global_step/sec: 615.819
INFO:tensorflow:loss = 6.59396, step = 2301 (0.162 sec)
INFO:tensorflow:global_step/sec: 644.025
INFO:tensorflow:loss = 2.78558, step = 2401 (0.155 sec)
INFO:tensorflow:global_step/sec: 617.206
INFO:tensorflow:loss = 2.65614, step = 2501 (0.162 sec)
INFO:tensorflow:global_step/sec: 620.459
INFO:tensorflow:loss = 3.418, step = 2601 (0.161 sec)
INFO:tensorflow:global_step/sec: 627.872
INFO:tensorflow:loss = 7.83943, step = 2701 (0.159 sec)
INFO:tensorflow:global_step/sec: 629.243
INFO:tensorflow:loss = 6.91227, step = 2

In [14]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x':np.array(test_set.data)},
    y=np.array(test_set.target),
    num_epochs=1,
    shuffle=False
)

accuracy_score = classifier.evaluate(input_fn=test_input_fn)['accuracy']
print('\nTest Accuracy: {0:f}\n'.format(accuracy_score))

INFO:tensorflow:Starting evaluation at 2017-11-08-23:35:25
INFO:tensorflow:Restoring parameters from output/iris_model/model.ckpt-6000
INFO:tensorflow:Finished evaluation at 2017-11-08-23:35:25
INFO:tensorflow:Saving dict for global step 6000: accuracy = 0.966667, average_loss = 0.0655332, global_step = 6000, loss = 1.966

Test Accuracy: 0.966667



In [18]:
new_samples = np.array(
    [[6.4,3.2,4.5,1.5],
    [5.8,3.1,5.0,1.7]], 
    dtype=np.float32
)
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x':new_samples},
    num_epochs=1,
    shuffle=False
)

predictions = list(classifier.predict(input_fn=predict_input_fn))
predicted_classes = [str(p['classes']) for p in predictions]

print('New Samples, Class Predictions: {}\n'.format(predicted_classes))

INFO:tensorflow:Restoring parameters from output/iris_model/model.ckpt-6000
New Samples, Class Predictions: ["[b'1']", "[b'2']"]

